In [1]:
import numpy as np
import pandas as pd 
from pprint import pprint
from collections import defaultdict

In [3]:
df = pd.read_csv('breakfast.csv')
data = df.Food.to_list()
data[:5]

['Bánh mì', 'Bánh mì', 'Cơm tấm', 'Phở', 'Súp cua']

$$
P(W_t|W_{t-1}) = \frac{Count("W_{t-1}W_t")}{Count("W_{t-1}")}
$$

trước tiên ta phải đếm các cặp và các giá trị đơn lẻ trước, sau đó đi chuẩn hóa

In [4]:
food_count = defaultdict(int)
food_pair_count = defaultdict(lambda: defaultdict(float))

In [5]:
# food_count: đếm số lần xuất hiện của một trạng thái
# food_pair_count: đếm tất cả các cặp trạng thái có thể [current][future]
n = len(data)
for i in range(n):
    food_count[data[i]] += 1
    if i == n - 1:
        # self loop
        food_pair_count[data[i]][data[i]] += 1
        break
    food_pair_count[data[i]][data[i + 1]] += 1

In [6]:
food_count

defaultdict(int, {'Bánh mì': 79, 'Cơm tấm': 91, 'Phở': 91, 'Súp cua': 104})

In [7]:
food_pair_count

defaultdict(<function __main__.<lambda>()>,
            {'Bánh mì': defaultdict(float,
                         {'Bánh mì': 21.0,
                          'Cơm tấm': 21.0,
                          'Súp cua': 21.0,
                          'Phở': 16.0}),
             'Cơm tấm': defaultdict(float,
                         {'Phở': 23.0,
                          'Bánh mì': 19.0,
                          'Súp cua': 22.0,
                          'Cơm tấm': 27.0}),
             'Phở': defaultdict(float,
                         {'Súp cua': 26.0,
                          'Cơm tấm': 23.0,
                          'Phở': 26.0,
                          'Bánh mì': 16.0}),
             'Súp cua': defaultdict(float,
                         {'Phở': 27.0,
                          'Súp cua': 35.0,
                          'Bánh mì': 22.0,
                          'Cơm tấm': 20.0})})

In [8]:
# chuẩn hóa theo tổng hàng
for key, value in food_pair_count.items():
    for k, v in value.items():
        food_pair_count[key][k] /= food_count[key] # chuẩn hóa

In [9]:
food_pair_count

defaultdict(<function __main__.<lambda>()>,
            {'Bánh mì': defaultdict(float,
                         {'Bánh mì': 0.26582278481012656,
                          'Cơm tấm': 0.26582278481012656,
                          'Súp cua': 0.26582278481012656,
                          'Phở': 0.20253164556962025}),
             'Cơm tấm': defaultdict(float,
                         {'Phở': 0.25274725274725274,
                          'Bánh mì': 0.2087912087912088,
                          'Súp cua': 0.24175824175824176,
                          'Cơm tấm': 0.2967032967032967}),
             'Phở': defaultdict(float,
                         {'Súp cua': 0.2857142857142857,
                          'Cơm tấm': 0.25274725274725274,
                          'Phở': 0.2857142857142857,
                          'Bánh mì': 0.17582417582417584}),
             'Súp cua': defaultdict(float,
                         {'Phở': 0.25961538461538464,
                          'Súp cua': 0.336538461

In [10]:
# lấy index của các món ăn để dễ thao tác
keys = list(food_count.keys())
idx = range(len(keys))
key_to_idx = dict(zip(keys, idx)) # key to index
print(key_to_idx)

{'Bánh mì': 0, 'Cơm tấm': 1, 'Phở': 2, 'Súp cua': 3}


Ta bây giờ có thể tạo ma trận 
 từ xác suất đã chuẩn hóa từ bước trên, và nên chuyển từ list sang numpy để tiện lợi cho việc tính toán hơn, do numpy là một thư viện rất mạnh của Python trong việc xử lý các thao tác liên quan đến đại số tuyến tính.

In [12]:
P = []
for key, value in food_pair_count.items():
    P.append(list(value.values()))
        
# chuyển list sang numpy để dễ tính toán
P = np.array(P)

print('Ma trận chuyển trạng thái P: ')
pprint(P)

Ma trận chuyển trạng thái P: 
array([[0.26582278, 0.26582278, 0.26582278, 0.20253165],
       [0.25274725, 0.20879121, 0.24175824, 0.2967033 ],
       [0.28571429, 0.25274725, 0.28571429, 0.17582418],
       [0.25961538, 0.33653846, 0.21153846, 0.19230769]])


In [13]:
# dự đoán món ăn 
curr_food = data[-1]
curr_distribution = P[key_to_idx[curr_food]]
predicted_food = np.random.choice(keys, p=curr_distribution) # random walk with known distribution
predicted_probability = P[key_to_idx[curr_food]][key_to_idx[predicted_food]]

In [15]:
predicted_probability

0.17582417582417584

In [16]:
# In ra kết quả dự đoán
print(f'Món ăn chúng ta ăn hôm trước: {data[-1]}')
print(f'Món ăn nên ăn vào hôm nay là "{predicted_food}"\
 với khả năng xảy ra là {round(predicted_probability * 100, 2)}%')


Món ăn chúng ta ăn hôm trước: Phở
Món ăn nên ăn vào hôm nay là "Súp cua" với khả năng xảy ra là 17.58%
